# MNIST 手寫數字辨識 (MNIST_RNN_TF)

2017/07/08
徐仕杰

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas



## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. 開始Deep Learning - RNN](#開始Deep Learning)  
  -  [A. 定義參數](#定義參數) 
  -  [B. 設計一個Model從X預測Y](#設計一個) 
  -  [C. 選一個loss function,](#選一個loss) 
  -  [D. 選一個optimizer](#選一個o) 
  -  [E. 開始執行訓練](#開始執行) 
  -  [F. 算一下正確率](#算一下正)  
<br>
-  [3. Reference](#reference)

<a id='import'></a>
## Import Package & Functions

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil

In [2]:
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [7]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. 開始Deep Learning - RNN
- What is RNN?![](img/rnn_2.png)
- Deep Learning ABC 
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [8]:
# hyperparameters
lr = 0.001 # learning rate
n_inputs = 28 # 每一行的维度
n_steps = 28 # 28 行
n_hidden_unins = 128 # 中間RNN的hidden units
n_classes = 10  # RNN最后的输出類別個数

In [9]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'in': tf.Variable(tf.random_uniform([n_inputs, n_hidden_unins], -1.0, 1.0), name="In_W"), # 输入层到中间层权重矩阵
    'out': tf.Variable(tf.random_uniform([n_hidden_unins, n_classes], -1.0, 1.0), name="Out_W"), # 中间层到输出层的权重
}
variable_summaries(W['in'],'In_W')
variable_summaries(W['out'],'Out_W')
B = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_unins]), name="In_Bias"), # 偏置
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes]), name="Out_Bias"),
}
variable_summaries(B['in'],'In_Bias')
variable_summaries(B['out'],'OUT_Bias')

#tfdot()  太亂了不看~~~

<a id='設計一個'></a>
###  B. 設計一個Model從X & X history預測Y  
$s(t)=f(Uw(t)+Ws(t-1)$  
$y(t)=g(Vs(t))$  
<br>
$w(t)$ : the current input   
$s(t-1)$ : the values in the hidden layer from the previous time step   
$s(t)$ : the current values hidden in the hidden layer    
$y$ : probability distribution of the picture   
$W, U$ : the weight matrices between input and hidden layer   
$V$ : the weight matrices between hidden and output layer   

![](img/rnn_3.png)


### L1. Input Layer 輸入層
- 第一步：: 本来数据的維度是(batch_size,28,28)但是這樣無法與權重Ｗ相乘, 因此需要轉為(-1,28), 這裡的-1表示任意長度, 就相當於把原來有batch_size這麼多張的圖片, 全部疊在一起
- 第二部：h=wx+b
- 第三步：後面RNN Cell那一層需要序列數據，因此又需要把數據變成(batch_size, 28, 128)的維度

In [10]:
# hidden_layer for input
# X : (128, 28, 28)
with tf.name_scope("inLayer"):
    X_ = tf.reshape(X, [-1, n_inputs])  # [batch_size*28, 28]
    X_in = tf.matmul(X_, W['in']) + B['in'] # (-1, 28) matmul (28, hidden_units) => (-1, hidden_units)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_unins]) # (batch_size, 28)


### L2. Hidden Layer 隱藏層
- 第一步：定義一個LSTM Cell
    - Base interface for all RNN Cells
        - tf.contrib.rnn.RNNCell
    - Core RNN Cells for use with TensorFlow's core RNN methods
        - tf.contrib.rnn.BasicRNNCell
        - tf.contrib.rnn.BasicLSTMCell
        - tf.contrib.rnn.GRUCell
        - tf.contrib.rnn.LSTMCell
        - tf.contrib.rnn.LayerNormBasicLSTMCell
- 第二步：把input輸入rnn cell


In [11]:
with tf.name_scope("RNN_CELL"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_unins) 
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(lstm_cell, X_in, dtype=tf.float32) 
    

### L3. Output Layer 輸出層
- states只存最後的結果, 因為是LSTM, 所以分別人了long,short兩條線的結果, 這邊直接取state[1]
- outputs包含了每一步的輸出, 這邊我們需要的是最後一步, 所以需要先把outputs tensor解開(unpack)變成列表[(batch, outputs)..] * steps
- 上述取其中一種方法就好

In [12]:
with tf.name_scope('outlayer'):
    pred = tf.matmul(rnn_states[1], W['out']) + B['out']
    #rnn_outputs = tf.unstack(tf.transpose(rnn_outputs, [1,0,2]))  
    #pred = tf.matmul(rnn_outputs[-1], weights['out']) + biases['out'] 
    
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
- cross_entorpy $-log(\Pr(Y_{true}))$

In [13]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_))

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B
- 就先用我們上次講的梯度下降Optimizer吧, 總之可以幫我們找到最小的cost function就好了~

In [14]:
optimizer = tf.train.AdamOptimizer(lr).minimize(loss) 
#optimizer = tf.train.AdagradOptimizer(lr).minimize(loss)

<a id='開始執行'></a>
### E. 開始執行訓練(Training Data + Validataion Data)

In [15]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
with tf.name_scope('performance'):  
    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [16]:
#先創造一個session, 然後記得要init variable
sess = tf.InteractiveSession()
sess.run(init)

if not os.path.exists('tb-log_5'):
    os.mkdir('tb-log_5')
summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

(50000, 784)

In [17]:
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        batch_x = batch_x.reshape([batch_size, n_inputs, n_steps])
        _, loss_s, acc_s, summary= sess.run([optimizer, loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_x, Y_:batch_y})
        summary_writer_train.add_summary(summary, ep * total_batch + i)
        if i%100 ==0:
            batch_vx = validation_X.reshape([-1, n_inputs, n_steps])
            loss_s, acc_s, summary= sess.run([loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_vx , Y_: validation_Y})
            acc = accuracy.eval({X: batch_vx , Y_: validation_Y})
            los=loss.eval({X: batch_vx , Y_: validation_Y})
            summary_writer_validation.add_summary(summary, ep * total_batch + i)
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc,x3=los,x4=i))
    print()


epoch:0, batch:300 loss:0.1724073588848114, acc:0.9465000033378601
epoch:1, batch:300 loss:0.10900198668241501, acc:0.9667000174522446
epoch:2, batch:300 loss:0.0846836268901825, acc:0.97320002317428598
epoch:3, batch:300 loss:0.0741218775510788, acc:0.97839999198913576
epoch:4, batch:300 loss:0.06407832354307175, acc:0.9804999828338623
epoch:5, batch:300 loss:0.06771568953990936, acc:0.9801999926567078
epoch:6, batch:300 loss:0.08114008605480194, acc:0.9757999777793884
epoch:7, batch:300 loss:0.06521962583065033, acc:0.9824000000953674
epoch:8, batch:300 loss:0.06359043717384338, acc:0.9836999773979187
epoch:9, batch:300 loss:0.06809202581644058, acc:0.9835000038146973
epoch:10, batch:300 loss:0.05826671048998833, acc:0.98540002107620246
epoch:11, batch:300 loss:0.05615396052598953, acc:0.9864000082015991
epoch:12, batch:300 loss:0.06187184900045395, acc:0.98570001125335697
epoch:13, batch:300 loss:0.0685330331325531, acc:0.98329997062683102
epoch:14, batch:300 loss:0.0579714179039001

In [23]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
_ = train_X[21:30].reshape([-1, n_inputs, n_steps])
pred_tmp = pred.eval(feed_dict={X: _ })
pred_tmp_df = pd.DataFrame(pred_tmp)

In [24]:

_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])

_ = train_X[21:30].reshape([-1, n_inputs, n_steps])
_ = correct_pred.eval({X: _ , Y_: train_Y[21:30]})
print('第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [0 9 1 1 2 4 3 2 7]


answer:  [0 9 1 1 2 4 3 2 7]
第21~30筆資料的比對結果:       0
0  True
1  True
2  True
3  True
4  True
5  True
6  True
7  True
8  True


<a id='算一下正'></a>
### F. 算一下正確率Accuracy (用Testing Data)
- unseen data
- generalize from the training set to the test set
- 泛化

In [25]:
# predict all X and get the accuracy
_ = test_X.reshape([-1, n_inputs, n_steps])
_ = accuracy.eval({X: _ , Y_: test_Y})
print('Accuracy:',_)

Accuracy: 0.9843


In [21]:
#!tensorboard --logdir=tb-log1 --port=6006

In [22]:
#sess.close()

<a id='reference'></a>
## Reference
- Statistical Language Models Based on Neural. Networks. Tomáš Mikolov. Ph. D. thesis, Brno University of Technology: http://www.fit.vutbr.cz/~imikolov/rnnlm/thesis.pdf
- Implementation Source Code: http://shomy.top/2017/01/02/tensorflow-rnn-example-3/
- youtube教學：https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-08-RNN2/